# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [215]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [216]:
data = pd.read_csv('customer_product_sales.csv')

In [217]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [218]:
# Uno los registros según el producto comprado por cada cliente, sumando las cantidades.

mydataframe = data.groupby(["CustomerID", "ProductName"]).agg({'Quantity':'sum'})
mydataframe.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [219]:
# Hago un pivot table para comparar todos los productos con todos los  clientes, según las cantidades compradas.

mydataframe2 = pd.pivot_table(mydataframe, values = "Quantity", index = "ProductName", columns = "CustomerID", 
                              fill_value = 0)
mydataframe2.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [220]:
# Hago una similarity matrix para obtener las similitudes entre los clientes. 

from scipy.spatial.distance import pdist, squareform

sim_matrix = squareform(pdist(mydataframe2.T, "euclidean"))

sim_matrix[:5]

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       [ 11.22497216,  12.08304597,  11.48912529, ..., 228.09866286,
        238.49318649, 229.46459422],
       [ 11.40175425,  12.56980509,  11.22497216, ..., 227.99780701,
        238.39672817, 228.92793626]])

In [221]:
# Creo una lista con los nombres de todos los clientes, para indexar abajo el dataframe 
# de las similitudes entre clientes. 

customerIDs = mydataframe2.columns.values

In [222]:
# Creo un dataframe a partir de la similarity matrix tipificada.

similaritydf = pd.DataFrame(1/(1 + squareform(pdist(mydataframe2.T, "euclidean"))), 
                         index = customerIDs, columns = customerIDs)

display(similaritydf.head())

,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [223]:
# Obtengo los 5 clientes más parecidos al cliente 33.

similarities = pd.DataFrame(similaritydf[33].sort_values(ascending=False)[1:6])
display(similarities)

,33
264,0.087047
3535,0.087047
3317,0.087047
2503,0.085983
3305,0.085638


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [224]:
# Filtro mi dataframe inicial por los registros de los 5 usuarios parecidos al cliente 33

sim_custs = data[data["CustomerID"].isin(similarities.index)]
sim_custs.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
159,264,Virgil,Small,24688,270,Cheese Cloth No 100,1
242,3535,Lana,Peters,2399506,270,Cheese Cloth No 100,1
887,3305,Bernard,O'Neill,2025094,102,"Rosemary - Primerba, Paste",1
1038,2503,Kyle,Reeves,182355,417,"Lamb - Pieces, Diced",1
1042,3535,Lana,Peters,4503728,417,"Lamb - Pieces, Diced",1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [225]:
# Uno los registros del dataframe filtrado, según los productos, sumando y ordenando las 
# cantidades de mayor a menor.

sim_cust2 = pd.DataFrame(sim_custs.groupby("ProductName").agg(
    {'Quantity':'sum'}).sort_values('Quantity', ascending = False)).reset_index()

sim_cust2.head()

,ProductName,Quantity
0,Butter - Unsalted,3
1,Wine - Ej Gallo Sierra Valley,3
2,Towels - Paper / Kraft,3
3,Soup - Campbells Bean Medley,3
4,Wine - Blue Nun Qualitatswein,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [226]:
mydataframe2.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


In [227]:
# Agrego al dataframe comparativo entre productos totales y clientes totales una columna 
# con las cantidades de los productos comprados por los clientes parecidos al 33.

m = sim_cust2.merge(mydataframe2, left_on = "ProductName", right_on = "ProductName")
m2 = m[["ProductName", "Quantity", 33]]
m2.head()

,ProductName,Quantity,33
0,Butter - Unsalted,3,0
1,Wine - Ej Gallo Sierra Valley,3,0
2,Towels - Paper / Kraft,3,1
3,Soup - Campbells Bean Medley,3,0
4,Wine - Blue Nun Qualitatswein,3,0


In [228]:
# Filtro los registros por los productos no comprados por el 33, y obtengo los 5 más populares,
# para recomendárselos.

sim_cust2_no33 = m2[m2[33] == 0]

top5 = list(m2["ProductName"][:5])
top5

['Butter - Unsalted',
 'Wine - Ej Gallo Sierra Valley',
 'Towels - Paper / Kraft',
 'Soup - Campbells Bean Medley',
 'Wine - Blue Nun Qualitatswein']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [229]:
# Creo una función para aplicar el mismo proceso a todos los clientes de mi dataset.

recommendations = {}

def recommended_products(customer):
    
    # Encuentro los 5 usuarios más similares:
    similarities = pd.DataFrame(similaritydf[customer].sort_values(ascending=False)[1:6])

    # Creo el dataframe con los registros de esos usuarios:
    sim_custs = data[data["CustomerID"].isin(similarities.index)]

    # Agrego los registros por producto, para obtener los productos más populares entre esos usuarios
    sim_cust2 = pd.DataFrame(sim_custs.groupby("ProductName").agg(
        {'Quantity':'sum'}).sort_values('Quantity', ascending = False)).reset_index()

    # Hago un merge de los productos comprados por los 5 usuarios, y los comprados por mi usuario, 
    # para saber cuáles le faltan de los que son populares entre los usuarios similares a él.
    m = sim_cust2.merge(mydataframe2, left_on = "ProductName", right_on = "ProductName")

    # Filtro el dataframe por los productos que mi usuario aun no tiene.
    sim_cust2_no33 = m[m[customer] == 0]

    # Obtengo los 5 productos más recomendados.
    top5 = list(m["ProductName"][:5])
    
    recommendations[customer] = top5

In [230]:
customers = list(set(data["CustomerID"]))
    
for customer in customers:
    recommended_products(customer)

In [231]:
# Compruebo varios resultados

print(recommendations[33])
print(recommendations[83973])

['Butter - Unsalted', 'Wine - Ej Gallo Sierra Valley', 'Towels - Paper / Kraft', 'Soup - Campbells Bean Medley', 'Wine - Blue Nun Qualitatswein']
['Baking Powder', 'Guinea Fowl', 'Cocoa Butter', 'Thermometer Digital', 'Muffin Chocolate Individual Wrap']


##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a row for each of the 5 product recommendations for each customer.

In [232]:
# Creo un dataframe que muestre los 5 productos recomendados por cada cliente, a partir del 
# diccionario obtenido arriba.

recommendationsdf = pd.DataFrame(recommendations)
recommendationsdf

,83973,59399,92168,49159,18441,22536,86028,75791,96272,32785,...,55281,75762,45042,59379,77810,77814,32753,20476,67582,65535
0,Baking Powder,Beer - Rickards Red,Bread - Italian Roll With Herbs,Cinnamon Buns Sticky,Rabbit - Whole,Kellogs All Bran Bars,"Water - Mineral, Natural","Pepsi - Diet, 355 Ml",Sea Bass - Whole,Barramundi,...,Sea Bass - Whole,Ice Cream Bar - Oreo Cone,Sea Bass - Whole,Pecan Raisin - Tarts,Olive - Spread Tapenade,Cheese - Victor Et Berthold,Grouper - Fresh,Pork - Kidney,Puree - Passion Fruit,Appetizer - Mushroom Tart
1,Guinea Fowl,"Shrimp - Baby, Warm Water","Bar Mix - Pina Colada, 355 Ml","Pasta - Penne, Rigate, Dry",Juice - Apple Cider,Appetizer - Sausage Rolls,Salmon Steak - Cohoe 8 Oz,Rice - Long Grain,Cocoa Butter,Cookies - Assorted,...,Pastry - Choclate Baked,"Wine - White, Mosel Gold",Spinach - Baby,Cod - Black Whole Fillet,Water - Aquafina Vitamin,Wiberg Super Cure,Sherry - Dry,Soup - Campbells Tomato Ravioli,Cake - Mini Cheesecake,V8 - Berry Blend
2,Cocoa Butter,Assorted Desserts,Cod - Black Whole Fillet,Cheese Cloth No 100,Soupfoamcont12oz 112con,Bacardi Breezer - Tropical,Spinach - Baby,Blueberries,Pears - Bosc,Beef - Rib Eye Aaa,...,Lettuce - California Mix,Sobe - Tropical Energy,Ice Cream Bar - Oreo Cone,Thermometer Digital,Pants Custom Dry Clean,"Lamb - Whole, Fresh","Yogurt - Blueberry, 175 Gr","Pasta - Detalini, White, Fresh",Ecolab - Solid Fusion,Fenngreek Seed
3,Thermometer Digital,Knife Plastic - White,Cup - Translucent 7 Oz Clear,"Oil - Shortening,liqud, Fry",Anchovy Paste - 56 G Tube,Cheese - Mozzarella,"Cheese - Brie, Triple Creme",Mussels - Cultivated,Sausage - Liver,"Pasta - Penne, Rigate, Dry",...,Wine - Redchard Merritt,Soupcontfoam16oz 116con,Onion Powder,"Pork - Back, Short Cut, Boneless",Sword Pick Asst,Oil - Safflower,"Chestnuts - Whole,canned","Bar Mix - Pina Colada, 355 Ml",Wine - Redchard Merritt,Sauce - Rosee
4,Muffin Chocolate Individual Wrap,Towels - Paper / Kraft,Bacardi Breezer - Tropical,Cookies - Assorted,Sprouts - Baby Pea Tendrils,Cheese Cloth No 100,Wine - Vineland Estate Semi - Dry,Wine - Ruffino Chianti,Clam Nectar,Table Cloth 54x72 White,...,V8 - Berry Blend,Vol Au Vents,Beets - Mini Golden,Beef Wellington,Cake - Box Window 10x10x2.5,Cake - Cake Sheet Macaroon,Napkin White - Starched,"Cheese - Boursin, Garlic / Herbs",Mussels - Frozen,"Lemonade - Natural, 591 Ml"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [233]:
# Creo un diccionario vacío donde voy a meter los distintos diccionarios de recomendaciones según
# el tipo de distancia entre clientes.

recommendations = {}

In [234]:
# Defino una función para crear similarity matrixes según el tipo de distancia entre clientes.

def similarity_matrix(distance):
    sim = pd.DataFrame(1/(1 + squareform(pdist(mydataframe2.T, distance))), 
                         index = customerIDs, columns = customerIDs)
    recommendations[distance] = {}
    return sim

In [235]:
# Defino una función para rellenar los diccionarios de recomendaciones,
# adaptada para recibir no solo distintos clientes, sino distintas 
# similarity matrix. No meto la creación de las similarity matrix en la función porque
# no tengo que crear una para cada cliente, solo 5.

def recommended_products(customer, sim, distance):
    similarities = pd.DataFrame(sim[customer].sort_values(ascending=False)[1:6])
    sim_custs = data[data["CustomerID"].isin(similarities.index)]
    sim_cust2 = pd.DataFrame(sim_custs.groupby("ProductName").agg(
        {'Quantity':'sum'}).sort_values('Quantity', ascending = False)).reset_index()
    m = sim_cust2.merge(mydataframe2, left_on = "ProductName", right_on = "ProductName")
    sim_cust2_no33 = m[m[customer] == 0]
    top5 = list(m["ProductName"][:5])
    recommendations[distance][customer] = top5

In [236]:
# Creo listas de clientes y de tipos de distancias

customers = list(set(data["CustomerID"]))

distances = ["euclidean", "correlation", "cityblock", "cosine", "jaccard"]

In [237]:
# Llamo a las dos funciones seguidas.

for dist in distances:
    sim = similarity_matrix(dist)
    for customer in customers:
        recommended_products(customer, sim, dist)

In [238]:
for dist in distances:
    print(dist, ":")
    display(pd.DataFrame(recommendations[dist]))

euclidean :


,83973,59399,92168,49159,18441,22536,86028,75791,96272,32785,...,55281,75762,45042,59379,77810,77814,32753,20476,67582,65535
0,Baking Powder,Beer - Rickards Red,Bread - Italian Roll With Herbs,Cinnamon Buns Sticky,Rabbit - Whole,Kellogs All Bran Bars,"Water - Mineral, Natural","Pepsi - Diet, 355 Ml",Sea Bass - Whole,Barramundi,...,Sea Bass - Whole,Ice Cream Bar - Oreo Cone,Sea Bass - Whole,Pecan Raisin - Tarts,Olive - Spread Tapenade,Cheese - Victor Et Berthold,Grouper - Fresh,Pork - Kidney,Puree - Passion Fruit,Appetizer - Mushroom Tart
1,Guinea Fowl,"Shrimp - Baby, Warm Water","Bar Mix - Pina Colada, 355 Ml","Pasta - Penne, Rigate, Dry",Juice - Apple Cider,Appetizer - Sausage Rolls,Salmon Steak - Cohoe 8 Oz,Rice - Long Grain,Cocoa Butter,Cookies - Assorted,...,Pastry - Choclate Baked,"Wine - White, Mosel Gold",Spinach - Baby,Cod - Black Whole Fillet,Water - Aquafina Vitamin,Wiberg Super Cure,Sherry - Dry,Soup - Campbells Tomato Ravioli,Cake - Mini Cheesecake,V8 - Berry Blend
2,Cocoa Butter,Assorted Desserts,Cod - Black Whole Fillet,Cheese Cloth No 100,Soupfoamcont12oz 112con,Bacardi Breezer - Tropical,Spinach - Baby,Blueberries,Pears - Bosc,Beef - Rib Eye Aaa,...,Lettuce - California Mix,Sobe - Tropical Energy,Ice Cream Bar - Oreo Cone,Thermometer Digital,Pants Custom Dry Clean,"Lamb - Whole, Fresh","Yogurt - Blueberry, 175 Gr","Pasta - Detalini, White, Fresh",Ecolab - Solid Fusion,Fenngreek Seed
3,Thermometer Digital,Knife Plastic - White,Cup - Translucent 7 Oz Clear,"Oil - Shortening,liqud, Fry",Anchovy Paste - 56 G Tube,Cheese - Mozzarella,"Cheese - Brie, Triple Creme",Mussels - Cultivated,Sausage - Liver,"Pasta - Penne, Rigate, Dry",...,Wine - Redchard Merritt,Soupcontfoam16oz 116con,Onion Powder,"Pork - Back, Short Cut, Boneless",Sword Pick Asst,Oil - Safflower,"Chestnuts - Whole,canned","Bar Mix - Pina Colada, 355 Ml",Wine - Redchard Merritt,Sauce - Rosee
4,Muffin Chocolate Individual Wrap,Towels - Paper / Kraft,Bacardi Breezer - Tropical,Cookies - Assorted,Sprouts - Baby Pea Tendrils,Cheese Cloth No 100,Wine - Vineland Estate Semi - Dry,Wine - Ruffino Chianti,Clam Nectar,Table Cloth 54x72 White,...,V8 - Berry Blend,Vol Au Vents,Beets - Mini Golden,Beef Wellington,Cake - Box Window 10x10x2.5,Cake - Cake Sheet Macaroon,Napkin White - Starched,"Cheese - Boursin, Garlic / Herbs",Mussels - Frozen,"Lemonade - Natural, 591 Ml"


correlation :


,83973,59399,92168,49159,18441,22536,86028,75791,96272,32785,...,55281,75762,45042,59379,77810,77814,32753,20476,67582,65535
0,Wine - White Cab Sauv.on,Beer - Rickards Red,Olive - Spread Tapenade,Wine - White Cab Sauv.on,V8 - Berry Blend,Mustard - Seed,Mayonnaise - Individual Pkg,Quiche Assorted,Phyllo Dough,Mustard - Seed,...,Lambcasing,Cheese - Wine,Tea - Decaf Lipton,"Sole - Dover, Whole, Fresh",Water - Aquafina Vitamin,Wine - Crozes Hermitage E.,Yogurt - French Vanilla,Coffee - Dark Roast,"Mushroom - Trumpet, Dry",Pastry - Butterscotch Baked
1,Browning Caramel Glace,Anchovy Paste - 56 G Tube,Pasta - Orecchiette,Wine - Chablis 2003 Champs,Whmis - Spray Bottle Trigger,Pail With Metal Handle 16l White,Mustard - Seed,Wine - Chablis 2003 Champs,Bread - Rye,Pail With Metal Handle 16l White,...,"Cheese - Brie, Triple Creme",Pasta - Angel Hair,Pail With Metal Handle 16l White,Cake - Mini Cheesecake,Gloves - Goldtouch Disposable,Cheese - Cottage Cheese,Garlic - Elephant,"Wine - Red, Colio Cabernet","Oil - Shortening,liqud, Fry",Clam Nectar
2,Tea - Herbal Sweet Dreams,Sword Pick Asst,Wanton Wrap,"Turnip - White, Organic",Halibut - Steaks,Mayonnaise - Individual Pkg,Pail With Metal Handle 16l White,Peas - Frozen,"Liners - Banana, Paper",Wine - Redchard Merritt,...,Sun - Dried Tomatoes,"Mushroom - Porcini, Dry",Mussels - Frozen,Ecolab - Mikroklene 4/4 L,Cookie Chocolate Chip With,"Soup - Campbells, Cream Of",Tia Maria,Beer - Blue,"Cheese - Brie,danish","Crab - Dungeness, Whole"
3,Pastry - Choclate Baked,Ezy Change Mophandle,Flavouring - Orange,Guinea Fowl,"Nut - Chestnuts, Whole",Hersey Shakes,Bandage - Flexible Neon,"Placemat - Scallop, White",Apricots - Dried,Arizona - Green Tea,...,Browning Caramel Glace,Dc - Frozen Momji,Longos - Grilled Chicken With,Water - Aquafina Vitamin,"Pork - Loin, Bone - In",Hersey Shakes,Durian Fruit,Muffin Mix - Blueberry,Beer - Rickards Red,Appetizer - Mushroom Tart
4,Rice - Jasmine Sented,Garlic - Peeled,Wiberg Super Cure,Cinnamon Buns Sticky,Wasabi Powder,Bread Crumbs - Panko,Initation Crab Meat,Beef - Ground Medium,Crush - Cream Soda,Pastry - Choclate Baked,...,Sardines,Loquat,Thermometer Digital,"Cheese - Brie, Triple Creme",Beef Ground Medium,"Wine - White, Colubia Cresh",Rice - Jasmine Sented,"Beer - Alexander Kieths, Pale Ale",Sword Pick Asst,Coffee - Irish Cream


cityblock :


,83973,59399,92168,49159,18441,22536,86028,75791,96272,32785,...,55281,75762,45042,59379,77810,77814,32753,20476,67582,65535
0,"Pork - Bacon, Double Smoked","Pepper - Black, Whole","Pepper - Black, Whole",Wine - Blue Nun Qualitatswein,Juice - Apple Cider,Kellogs All Bran Bars,"Sole - Dover, Whole, Fresh",Veal - Sweetbread,Soup - Campbells Bean Medley,Soup - Campbells Bean Medley,...,"Veal - Inside, Choice",Butter - Unsalted,Soupfoamcont12oz 112con,"Sole - Dover, Whole, Fresh",Scallops - 10/20,Wanton Wrap,Tia Maria,Pork - Kidney,Lettuce - Spring Mix,Knife Plastic - White
1,"Nut - Chestnuts, Whole","Lamb - Pieces, Diced",Wine - Blue Nun Qualitatswein,Butter - Unsalted,Rabbit - Whole,Knife Plastic - White,Knife Plastic - White,Butter - Unsalted,Butter - Unsalted,Butter - Unsalted,...,Pork - Kidney,Soup - Campbells Bean Medley,Bay Leaf,Squid - Tubes / Tenticles 10/20,Mayonnaise - Individual Pkg,Bandage - Fexible 1x3,Butter - Unsalted,"Pasta - Detalini, White, Fresh","Salsify, Organic",Butter - Unsalted
2,Olive - Spread Tapenade,Soupfoamcont12oz 112con,Muffin Batt - Blueberry Passion,Tia Maria,Anchovy Paste - 56 G Tube,Cheese - Mozzarella,Wine - Chardonnay South,Guinea Fowl,"Pepper - Black, Whole","Pepper - Black, Whole",...,"Cheese - Boursin, Garlic / Herbs",Ecolab - Solid Fusion,"Veal - Inside, Choice",Wine - Chardonnay South,Wanton Wrap,"Wine - Magnotta, Merlot Sr Vqa",Veal - Sweetbread,Knife Plastic - White,Pork - Kidney,Cake - Cake Sheet Macaroon
3,Rice - Jasmine Sented,Tahini Paste,Knife Plastic - White,Veal - Sweetbread,Beans - Wax,Butter - Unsalted,Foam Dinner Plate,Bar - Granola Trail Mix Fruit Nut,Ketchup - Tomato,Ketchup - Tomato,...,Pork - Hock And Feet Attached,Bay Leaf,Butter - Unsalted,"Pepper - White, Ground",Bay Leaf,"Lamb - Pieces, Diced",Tilapia - Fillets,"Pepper - Black, Whole","Veal - Brisket, Provimi,bnls",Pomello
4,Butter - Unsalted,"Pork - Bacon, Double Smoked",Chinese Foods - Chicken,Wine - Chardonnay South,Soup V8 Roasted Red Pepper,Appetizer - Sausage Rolls,Sauce - Hollandaise,Berry Brulee,Knife Plastic - White,Knife Plastic - White,...,Foam Cup 6 Oz,Knife Plastic - White,"Soup - Campbells, Beef Barley",Veal - Eye Of Round,"Lamb - Pieces, Diced",Squid - Tubes / Tenticles 10/20,Table Cloth 81x81 White,Wine - Ej Gallo Sierra Valley,Halibut - Steaks,Veal - Sweetbread


cosine :


,83973,59399,92168,49159,18441,22536,86028,75791,96272,32785,...,55281,75762,45042,59379,77810,77814,32753,20476,67582,65535
0,Wine - White Cab Sauv.on,Ezy Change Mophandle,Olive - Spread Tapenade,Wine - White Cab Sauv.on,V8 - Berry Blend,Mustard - Seed,Mayonnaise - Individual Pkg,Quiche Assorted,Phyllo Dough,Mustard - Seed,...,Sardines,"Wine - White, Mosel Gold",Tea - Decaf Lipton,"Sole - Dover, Whole, Fresh",Water - Aquafina Vitamin,Wine - Crozes Hermitage E.,Yogurt - French Vanilla,"Wine - Red, Colio Cabernet","Mushroom - Trumpet, Dry",Grapes - Red
1,Browning Caramel Glace,Anchovy Paste - 56 G Tube,Pasta - Orecchiette,Wine - Chablis 2003 Champs,Whmis - Spray Bottle Trigger,Pail With Metal Handle 16l White,Mustard - Seed,Wine - Chablis 2003 Champs,Pears - Bosc,Wine - Redchard Merritt,...,"Beans - Kidney, Canned",Cheese - Wine,Pail With Metal Handle 16l White,Cake - Mini Cheesecake,Veal - Osso Bucco,Hersey Shakes,Island Oasis - Mango Daiquiri,Coffee - Dark Roast,"Oil - Shortening,liqud, Fry",Clam Nectar
2,Tea - Herbal Sweet Dreams,Garlic - Peeled,Wanton Wrap,"Turnip - White, Organic",Halibut - Steaks,Mayonnaise - Individual Pkg,Pail With Metal Handle 16l White,Peas - Frozen,Pork - Belly Fresh,Pail With Metal Handle 16l White,...,Browning Caramel Glace,Mustard Prepared,Mussels - Frozen,Ecolab - Mikroklene 4/4 L,Garbag Bags - Black,"Soup - Campbells, Cream Of",Tia Maria,Bandage - Flexible Neon,"Cheese - Brie,danish","Crab - Dungeness, Whole"
3,Pastry - Choclate Baked,Pastry - Choclate Baked,Flavouring - Orange,Guinea Fowl,"Nut - Chestnuts, Whole",Hersey Shakes,Bandage - Flexible Neon,"Placemat - Scallop, White",Crackers Cheez It,Thermometer Digital,...,Sun - Dried Tomatoes,Loquat,Longos - Grilled Chicken With,Water - Aquafina Vitamin,Mangoes,Wine - Sogrape Mateus Rose,Cheese - Wine,Muffin Mix - Blueberry,Beer - Rickards Red,Pastry - Butterscotch Baked
4,Rice - Jasmine Sented,Broom - Corn,Wiberg Super Cure,Cinnamon Buns Sticky,Wasabi Powder,Muffin Batt - Choc Chk,Initation Crab Meat,Beef - Ground Medium,Fondant - Icing,Bread Crumbs - Panko,...,Juice - Apple Cider,Tomato - Tricolor Cherry,Thermometer Digital,"Cheese - Brie, Triple Creme",Milk Powder,Cheese - Cottage Cheese,Garlic - Elephant,Beer - Blue,Sword Pick Asst,Coffee - Irish Cream


jaccard :


,83973,59399,92168,49159,18441,22536,86028,75791,96272,32785,...,55281,75762,45042,59379,77810,77814,32753,20476,67582,65535
0,"Pepper - White, Ground","Beans - Kidney, Canned",Garbage Bags - Clear,Flavouring - Orange,Bread Foccacia Whole,Pants Custom Dry Clean,Blueberries,Beef - Top Sirloin - Aaa,Knife Plastic - White,Juice - Happy Planet,...,Coffee - Hazelnut Cream,Sherry - Dry,Crackers - Trio,Ecolab - Mikroklene 4/4 L,Beef - Montreal Smoked Brisket,Kiwi,Vaccum Bag 10x13,Puree - Mocha,Beets - Mini Golden,"Beans - Kidney, Canned"
1,Towels - Paper / Kraft,"Cheese - Brie, Triple Creme",Blackberries,Cookie Dough - Double,Macaroons - Two Bite Choc,Cocktail Napkin Blue,Lettuce - Treviso,Muffin - Zero Transfat,Vinegar - Sherry,"Placemat - Scallop, White",...,Anchovy Paste - 56 G Tube,Cheese - Wine,Dc - Frozen Momji,Broom - Corn,Veal - Inside,"Mushroom - Trumpet, Dry","Liners - Banana, Paper",Broom - Corn,"Lamb - Pieces, Diced",Tea - English Breakfast
2,Milk - 1%,"Rum - Coconut, Malibu",Wiberg Super Cure,Mussels - Cultivated,Bread - English Muffin,Puree - Passion Fruit,Bread - Calabrese Baguette,Beer - Labatt Blue,Zucchini - Yellow,Soup - Campbells Tomato Ravioli,...,Remy Red,Beef - Top Sirloin - Aaa,Bread - Italian Roll With Herbs,"Pepsi - Diet, 355 Ml",Knife Plastic - White,Bread - Calabrese Baguette,Whmis - Spray Bottle Trigger,Mustard - Seed,Pie Filling - Cherry,Wine - Chablis 2003 Champs
3,"Wine - Magnotta, Merlot Sr Vqa",Cheese - Parmesan Cubes,Salmon - Sockeye Raw,"Cup - 6oz, Foam","Juice - Cranberry, 341 Ml",Isomalt,Bread - French Baquette,Tia Maria,Langers - Ruby Red Grapfruit,Tomato - Tricolor Cherry,...,Clam Nectar,Beer - Blue,Halibut - Fletches,Baking Powder,Island Oasis - Mango Daiquiri,Cumin - Whole,"Tart Shells - Sweet, 4","Bread - Roll, Canadian Dinner","Beer - Alexander Kieths, Pale Ale","Liners - Banana, Paper"
4,"Chocolate - Semi Sweet, Calets",Gloves - Goldtouch Disposable,Cumin - Whole,Wine - White Cab Sauv.on,"Beef - Ground, Extra Lean, Fresh","Nut - Pistachio, Shelled","Appetizer - Mini Egg Roll, Shrimp",Sea Bass - Whole,Cookie Dough - Double,Chips Potato Salt Vinegar 43g,...,Wine - White Cab Sauv.on,Beef - Short Loin,Bread Fig And Almond,Coffee - Dark Roast,Mayonnaise - Individual Pkg,Beef - Montreal Smoked Brisket,"Lentils - Red, Dry","Pork - Loin, Bone - In",Rambutan,Berry Brulee
